In [121]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import math

In [122]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [123]:
# driver.get('https://annuairesante.ameli.fr/')
# inputsearch=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,'input-46')))
# searchbuttn=driver.find_element(By.XPATH, '//*[@data-testid="searchBtn"]')
# inputclass=driver.find_element(By.XPATH, '//*[@data-testid="searchProfessionInput"]')

In [124]:
list_of_cities=['angers','BREST','CAEN']

In [125]:
name=[]
adress=[]
numero=[]
speciality=[]
categories=[]



In [126]:
for city in list_of_cities:
    driver.get('https://annuairesante.ameli.fr/')
    time.sleep(2)
    inputsearch=WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,'input-46')))
    searchbuttn=driver.find_element(By.XPATH, '//*[@data-testid="searchBtn"]')
    inputclass=driver.find_element(By.XPATH, '//*[@data-testid="searchProfessionInput"]')
    inputsearch.clear()
    time.sleep(1)
    inputsearch.send_keys(city)
    time.sleep(1)
    time.sleep(2)
    optionsearch=driver.find_elements(By.CSS_SELECTOR, 'div[data-v-f8559e0c][role="menuitem"]')
    optionsearch[1].click()
    inputclass.send_keys('Ophtalmologiste')
    time.sleep(1)
    optionclass=driver.find_element(By.CSS_SELECTOR, 'div[class="v-list v-select-list v-sheet theme--light theme--light"]')
    optionclass.click()

    time.sleep(1)
    searchbuttn.click()
    descnum=0
    errorcouter=0
    while descnum==0:
        driver.execute_script(f"window.scrollBy(0, {15500});")
        time.sleep(1)
        page=BeautifulSoup(driver.page_source,'html.parser')
        alltargets=page.find_all('div',class_='card-holder-pointer')
        for targets in alltargets:
          try:
            prename=targets.find('p',class_='text-lg-subtitle-1 mb-0 primary--text').text
            if prename:
                name.append(prename)
            else:
                name.append(None)
          except AttributeError:
                name.append(None)
          try:
            prespeciality=targets.find('div',class_='d-block align-baseline').text
            if prespeciality:
                speciality.append(prespeciality)
            else:
                speciality.append(None)
          except AttributeError:
                speciality.append(None)
          try:
            precategories=targets.find('p',class_='grey-text text-body-2 mb-0 mt-1').text
            if precategories:
                categories.append(precategories)
            else:
                categories.append(None)
          except AttributeError:
                categories.append(None)
          try:
            preadress=targets.find('span',attrs={"data-testid": "adresse"}).text
            if preadress:
                adress.append(preadress)
            else:
                adress.append(None)
          except AttributeError:
                adress.append(None)        
          try:
            prenumero=targets.find('span',attrs={"data-testid": "telephone"}).text
            if prenumero:
                numero.append(prenumero)
            else:
                numero.append(None)
          except AttributeError:
                numero.append(None)    
    
    
    
    
        if len(driver.find_elements(By.XPATH,'//button[@aria-label="Page suivante"]'))>0:
            try:
                driver.find_element(By.XPATH,'//button[@aria-label="Page suivante"]').click()
                time.sleep(2)
            except ElementClickInterceptedException:
                time.sleep(2)
                errorcouter+=1
            if errorcouter>5:
                    descnum=1
        else:
            descnum=1    


KeyboardInterrupt: 

In [ ]:
len(driver.find_elements(By.XPATH,'//button[@aria-label="Page suivante"]'))

In [ ]:
sample=({
    'Name':name,
    'Number':numero,
    'Adress':adress,
    'Categorie':categories,
    'Speciality':speciality
})

In [ ]:
data=pd.DataFrame(sample).drop_duplicates()

In [ ]:
data

In [ ]:

data.to_csv('annuairesante_with_services_update.csv',index=False)
data.to_excel('annuairesante_with_services_update.xlsx',index=False)